# Plot all Nakazato 2013 models together

In [1]:
import pandas as pd
import plotly.express as px

from sspike.detectors import Detector
from sspike.env import sspike_dir

### Get file paths to tabulated results

In [2]:
sn_names = ['N13-13-04-100',
             'N13-13-04-200',
             'N13-13-04-300',
             'N13-13-20-100',
             'N13-13-20-200',
             'N13-13-20-300',
             'N13-20-04-100',
             'N13-20-04-200',
             'N13-20-04-300',
             'N13-20-20-100',
             'N13-20-20-200',
             'N13-20-20-300',
             'N13-30-20-100',
             'N13-30-20-200',
             'N13-30-20-300',
             'N13-50-04-100',
             'N13-50-04-200',
             'N13-50-04-300',
             'N13-50-20-100',
             'N13-50-20-200',
             'N13-50-20-300']
# sn_names = ['N13-20-20-300']
bin_dir = '10.0kpc-NT/bins1ti-0.05tf20.0'

### Read each file into a data frame

In [3]:
# Columns for simulation type and events per channel.
column_names = ['Mass', 'Metal', 'Revival [ms]', 'Channel', 'Counts']
Z = {'20': 'Solar', '04': 'SMC'}

row_list = []
# One row for each channel for each simulation.
for sn in sn_names:
    _, m, z, t = sn.split('-')

    path = f'{sspike_dir}/{sn}/{bin_dir}/totals.csv'

    df = pd.read_csv(path, sep=' ')
    keep = (df['file'] == 'smeared_weighted') | (df['channel'] == 'nc_p_cut')
    df_cut = df.where(keep).dropna().drop(columns='file').to_numpy()
    
    for row in df_cut:
        new_row = [m, Z[z], t] + row.tolist()
        row_list.append(new_row)

df = pd.DataFrame(row_list, columns=column_names)
df.replace('nc_p_cut', 'nc_p', inplace=True)

In [23]:
bars = px.bar(df, x='Revival [ms]', y='Counts', 
              color='Channel', barmode='group',
              facet_row='Metal', facet_col='Mass',
              category_orders={"Mass": ["13", "20", "30", "50"],
                               "Metal": ["Solar", "SMC"]},
              log_y=True)
bars.layout.bargroupgap = 0.2
bars.layout.font = dict(size=18, family="Times New Roman")
bars.layout.bargap = 0.1
bars.layout.bargroupgap = 0.05
bars.layout.width = 1100
bars.layout.height = 400
bars.layout.title = f'Nakazato_2013 @ 10 kpc in KamLAND'
bars

In [11]:
bars.write_image('../out/plots/nak_all.png', width=1100, height=400, scale=3)
bars.write_html('../out/plots/nak_all.html')